In [7]:
import os
from pathlib import Path

repo_root = Path.cwd()
while repo_root != repo_root.parent and not (repo_root / ".git").exists():
    repo_root = repo_root.parent

os.chdir(repo_root)
print(f"Running notebook from {repo_root}")

Running notebook from c:\Users\Aakil\Documents\GitHub\Data-cleanup-and-insights-portfolio


## 1. Load raw customer data

In [8]:
import pandas as pd
from pathlib import Path

DATA_DIR = Path("01-clean-dataset")
RAW_PATH = DATA_DIR / "raw_customers.csv"
CLEAN_PATH = DATA_DIR / "cleaned_customers.csv"

df = pd.read_csv(RAW_PATH)
raw_row_count = len(df)
df.head()

,customer_id,first_name,last_name,email,signup_date,country,age,newsletter_opt_in
0,CUST001,Anna,Jansen,anna.jansen@example.com,2023-01-14,Netherlands,34.0,yes
1,CUST002,Miguel,van Dijk,miguel.vandijk@example.com,2023-02-03,NL,28.0,no
2,CUST003,sophie,Visser,sophie.v@example.com,NaN,Netherlands,27.0,Y
3,CUST004,Lars,Bakker,lars.bakker@example.com,2022-11-22,nl,42.0,N
4,CUST005,Sara,Smits,sara.smits@example.com,2023-03-18,NETHERLANDS,5.0,yes


In [13]:
df.info()
df.describe(include=["number"])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   customer_id        33 non-null     object 
 1   first_name         33 non-null     object 
 2   last_name          33 non-null     object 
 3   email              33 non-null     object 
 4   signup_date        30 non-null     object 
 5   country            33 non-null     object 
 6   age                32 non-null     float64
 7   newsletter_opt_in  31 non-null     object 
dtypes: float64(1), object(7)
memory usage: 2.3+ KB


,age
count,32.000000
mean,39.031250
std,21.639993
min,0.000000
25%,27.750000
50%,35.500000
75%,45.500000
max,120.000000


## 2. Quick issues spotted
- Country values mix `Netherlands`, `NL`, `nl`, and uppercase variants.
- Names/emails contain stray spaces and inconsistent casing.
- Newsletter opt-in mixes `yes/no`, `Y/N`, and blanks.
- Ages include outliers (e.g., 5, 120) and missing entries.
- Duplicate emails exist, and one row is completely empty.
- Several signup dates are blank, so we should convert them to true missing values.

In [14]:
df = df.dropna(how="all").copy()

string_columns = df.select_dtypes(include="object").columns.tolist()
for column in string_columns:
    df[column] = df[column].str.strip()

df.replace({"": pd.NA}, inplace=True)
df["email"] = df["email"].str.lower()
df["signup_date"] = pd.to_datetime(df["signup_date"], errors="coerce")

def normalize_country(value):
    if pd.isna(value):
        return pd.NA
    token = str(value).strip().lower()
    token = token.replace(" ", "")
    if token in {"nl", "netherlands", "nederland"}:
        return "Netherlands"
    return token.title()

df["country"] = df["country"].apply(normalize_country)

def normalize_opt_in(value):
    if pd.isna(value):
        return pd.NA
    token = str(value).strip().lower()
    if token in {"y", "yes", "true", "1"}:
        return "Yes"
    if token in {"n", "no", "false", "0"}:
        return "No"
    return pd.NA

df["newsletter_opt_in"] = df["newsletter_opt_in"].apply(normalize_opt_in)

df["age"] = pd.to_numeric(df["age"], errors="coerce")
age_mask = df["age"].between(16, 90)
df.loc[~age_mask, "age"] = pd.NA
df["age"] = df["age"].astype("Int64")

df["first_name"] = df["first_name"].str.title()
df["last_name"] = df["last_name"].str.title()

before_dedup = len(df)
df = df.drop_duplicates(subset="email", keep="first")
dedup_removed = before_dedup - len(df)

df = df.dropna(subset=["customer_id", "email"])
df.reset_index(drop=True, inplace=True)

print(f"Rows in raw file: {raw_row_count}")
print(f"Rows after cleaning: {len(df)} (removed {raw_row_count - len(df)} rows)")
print(f"Duplicate emails removed: {dedup_removed}")

Rows in raw file: 34
Rows after cleaning: 32 (removed 2 rows)
Duplicate emails removed: 1


In [15]:
df.info()
df["country"].value_counts(dropna=False)
df["newsletter_opt_in"].value_counts(dropna=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   customer_id        32 non-null     object        
 1   first_name         32 non-null     object        
 2   last_name          32 non-null     object        
 3   email              32 non-null     object        
 4   signup_date        29 non-null     datetime64[ns]
 5   country            32 non-null     object        
 6   age                28 non-null     Int64         
 7   newsletter_opt_in  30 non-null     object        
dtypes: Int64(1), datetime64[ns](1), object(6)
memory usage: 2.2+ KB


newsletter_opt_in
Yes     15
No      15
<NA>     2
Name: count, dtype: int64

In [16]:
df.to_csv(CLEAN_PATH, index=False)
print(f"Clean file saved to {CLEAN_PATH}")

Clean file saved to 01-clean-dataset\cleaned_customers.csv


**Cleaning summary**
- Trimmed whitespace, lower-cased emails, and title-cased names.
- Normalised all country values to `Netherlands` and newsletter values to `Yes/No`.
- Converted signup dates to proper datetimes and nullified unrealistic ages.
- Removed blank rows plus duplicate emails before exporting `cleaned_customers.csv`.